In [123]:
from ctgan import CTGAN
from data_transformer import DataTransformer
from data_sampler import DataSampler
from train import Train
import numpy as np
import tensorflow as tf

In [124]:
import pandas as pd
df = pd.read_csv('tr_by_acct_w_age.csv')
df.sort_values(by = ["account_id", "date"])

,Unnamed: 0,column_a,account_id,date,type,operation,amount,balance,k_symbol,age,tcode
0,0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,29,cash_cr
1,1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,29,bank_cr
2,2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,29,cash_cr
3,3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,29,interest_cr
4,4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,29,bank_cr
...,...,...,...,...,...,...,...,...,...,...,...
1056315,1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,46,cash_db_nan
1056316,1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,46,bank_cr
1056317,1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,46,cash_db_nan
1056318,1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,46,interest_cr


In [125]:
from datetime import datetime
import calendar
czech_date_parser = lambda x: datetime.strptime(str(x), "%y%m%d")

df["datetime"] = df["date"].apply(czech_date_parser)

In [126]:
df["month"] = df["datetime"].dt.month
df["day"] = df["datetime"].dt.day
df["dow"] =  df["datetime"].dt.dayofweek
df["year"] = df["datetime"].dt.year

# dtme - days till month end
df["dtme"] = df.datetime.apply(lambda dt: calendar.monthrange(dt.year, dt.month)[1] - dt.day)

In [127]:
cat_code_fields = ['type', 'operation', 'k_symbol']
TCODE_SEP = "__"
# create tcode by concating fields in "cat_code_fields"
def set_tcode(df, cat_code_fields):
    tcode = df[cat_code_fields[0]].astype(str)
    for ccf in cat_code_fields[1:]:
        tcode += TCODE_SEP + df[ccf].astype(str)

    df["tcode"] = tcode
    
    
set_tcode(df, cat_code_fields)

In [128]:
conditions = [
    (df['day'] >= 1) & (df['day'] <= 10),
    (df['day'] > 10) & (df['day'] <= 20),
    (df['day'] > 20) & (df['day'] <= 31)
]
categories = ['first', 'middle', 'last']

# Use numpy.select() to map the numbers to categories
df['dtme_cat'] = np.select(conditions, categories)

In [129]:
bin_edges = [17, 30, 40, 50, 60, 81]
labels = ['18-30', '31-40', '41-50', '51-60', '61+']

# Use pd.cut() to convert ages to categorical groups
df['age_group'] = pd.cut(df['age'], bins=bin_edges, labels=labels, right=False)
df['age_group'] = df['age_group'].astype('object')

In [130]:
result = df.groupby('account_id')['datetime'].agg(['min', 'max'])
result['duration'] = result['max'] - result['min']
result_sorted = result.sort_values('duration', ascending=False)

In [131]:
df["td"] = df[["account_id", "datetime"]].groupby("account_id").diff()
df["td"] = df["td"].apply(lambda x: x.days)
df["td"].fillna(0.0, inplace=True)

In [132]:
df

,Unnamed: 0,column_a,account_id,date,type,operation,amount,balance,k_symbol,age,tcode,datetime,month,day,dow,year,dtme,dtme_cat,age_group,td
0,0,149432,1,950324,CREDIT,CREDIT IN CASH,1000.0,1000.0,NaN,29,CREDIT__CREDIT IN CASH__nan,1995-03-24,3,24,4,1995,7,last,18-30,0.0
1,1,157404,1,950413,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,4679.0,NaN,29,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-04-13,4,13,3,1995,17,middle,18-30,20.0
2,2,158832,1,950423,CREDIT,CREDIT IN CASH,12600.0,17279.0,NaN,29,CREDIT__CREDIT IN CASH__nan,1995-04-23,4,23,6,1995,7,last,18-30,10.0
3,3,162681,1,950430,CREDIT,NaN,19.2,17298.2,INTEREST CREDITED,29,CREDIT__nan__INTEREST CREDITED,1995-04-30,4,30,6,1995,0,last,18-30,7.0
4,4,167083,1,950513,CREDIT,COLLECTION FROM ANOTHER BANK,3679.0,20977.2,NaN,29,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-05-13,5,13,5,1995,18,middle,18-30,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,1056315,1033141,11382,981202,DEBIT,CASH WITHDRAWAL,25600.0,41114.4,NaN,46,DEBIT__CASH WITHDRAWAL__nan,1998-12-02,12,2,2,1998,29,first,41-50,2.0
1056316,1056316,1040574,11382,981210,CREDIT,COLLECTION FROM ANOTHER BANK,46248.0,87362.4,NaN,46,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1998-12-10,12,10,3,1998,21,first,41-50,8.0
1056317,1056317,1050362,11382,981225,DEBIT,CASH WITHDRAWAL,6300.0,81062.4,NaN,46,DEBIT__CASH WITHDRAWAL__nan,1998-12-25,12,25,4,1998,6,last,41-50,15.0
1056318,1056318,1053037,11382,981231,CREDIT,NaN,311.3,81373.6,INTEREST CREDITED,46,CREDIT__nan__INTEREST CREDITED,1998-12-31,12,31,3,1998,0,last,41-50,6.0


In [133]:
raw_data = df[['amount', 'balance', 'tcode', 'month', 'dow', 'year', 'dtme_cat', 'age_group', 'td']]

In [134]:
raw_data

,amount,balance,tcode,month,dow,year,dtme_cat,age_group,td
0,1000.0,1000.0,CREDIT__CREDIT IN CASH__nan,3,4,1995,last,18-30,0.0
1,3679.0,4679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,4,3,1995,middle,18-30,20.0
2,12600.0,17279.0,CREDIT__CREDIT IN CASH__nan,4,6,1995,last,18-30,10.0
3,19.2,17298.2,CREDIT__nan__INTEREST CREDITED,4,6,1995,last,18-30,7.0
4,3679.0,20977.2,CREDIT__COLLECTION FROM ANOTHER BANK__nan,5,5,1995,middle,18-30,13.0
...,...,...,...,...,...,...,...,...,...
1056315,25600.0,41114.4,DEBIT__CASH WITHDRAWAL__nan,12,2,1998,first,41-50,2.0
1056316,46248.0,87362.4,CREDIT__COLLECTION FROM ANOTHER BANK__nan,12,3,1998,first,41-50,8.0
1056317,6300.0,81062.4,DEBIT__CASH WITHDRAWAL__nan,12,4,1998,last,41-50,15.0
1056318,311.3,81373.6,CREDIT__nan__INTEREST CREDITED,12,3,1998,last,41-50,6.0


In [135]:
raw_data.dtypes

amount       float64
balance      float64
tcode         object
month          int64
dow            int64
year           int64
dtme_cat      object
age_group     object
td           float64
dtype: object

In [136]:
transformer = DataTransformer()
transformer.fit(raw_data, discrete_columns=('tcode', 'month', 'dow', 'year', 'dtme_cat', 'age_group'))
data_t = transformer.transform(raw_data)      #matrix of transformed data

In [139]:
data_t.shape

(1056320, 81)

In [142]:
df_restore = pd.DataFrame(data_t, columns=[f'v_{i}' for i in range(1, 82)])
df_restore['account_id'] = df['account_id']

In [143]:
df_restore

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,...,v_73,v_74,v_75,v_76,v_77,v_78,v_79,v_80,v_81,account_id
0,0.002995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.063758,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2,0.005030,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-0.197160,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,0.063758,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,0.327848,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11382
1056316,0.039862,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11382
1056317,-0.126785,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11382
1056318,-0.287540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11382


In [84]:
max_seq_len = 80
min_seq_len = 20

In [85]:
gb_aid = df.groupby("account_id")["account_id"]

full_seqs_per_acct = gb_aid.count() // max_seq_len

In [86]:
full_seqs_per_acct

account_id
1        2
2        5
3        1
4        2
5        1
        ..
11333    4
11349    3
11359    4
11362    4
11382    3
Name: account_id, Length: 4500, dtype: int64

In [87]:
n_full_seqs = sum(full_seqs_per_acct)

In [88]:
n_full_seqs

10995

In [89]:
n_part_seqs = sum(gb_aid.count() - full_seqs_per_acct*max_seq_len >= min_seq_len)
n_part_seqs

3359

In [90]:
n_seqs = n_full_seqs + n_part_seqs

In [93]:
n_feat_inp = 9
n_steps = max_seq_len = 80
inp_tensor = np.zeros((n_seqs, n_steps, n_feat_inp))

In [102]:
import time
seq_i = 0
rows_per_acct = {}
alert_every = 2000
attribute = "age_sc"


attributes = np.zeros(n_seqs)
start_time = time.time()
for acct_id, group in df.groupby("account_id"):
    rows_per_acct[acct_id] = []
    
    for i in range(len(group) // max_seq_len + 1):

        n_trs = len(group)
        start = i*max_seq_len
        seq_len = min(max_seq_len, n_trs - start)   

        if seq_len >= min_seq_len:
            inp_tensor[seq_i, :seq_len] = group.iloc[start:start+seq_len]
            attributes[seq_i] = group["age"].iloc[0]

            rows_per_acct[acct_id].append(seq_i)
            seq_i += 1
            
            if seq_i % alert_every == 0:
                print(f"Finished encoding {seq_i} of {n_seqs} seqs")
    
    
                
 
# Add conditioning info (attribute) to first timestep of inp
inp_tensor = np.concatenate([np.repeat(attributes[:, None, None], n_feat_inp, axis=2), 
                             inp_tensor], 
                             axis=1)

Finished encoding 2000 of 14354 seqs
Finished encoding 4000 of 14354 seqs
Finished encoding 6000 of 14354 seqs
Finished encoding 8000 of 14354 seqs
Finished encoding 10000 of 14354 seqs
Finished encoding 12000 of 14354 seqs
Finished encoding 14000 of 14354 seqs


In [32]:
grouped = df.groupby('account_id').apply(lambda x: x.iloc[0:80])
grouped[grouped['account_id'] == 1]

Unnamed: 0  column_a  account_id    date    type  \
account_id                                                        
1          0            0    149432           1  950324  CREDIT   
           1            1    157404           1  950413  CREDIT   
           2            2    158832           1  950423  CREDIT   
           3            3    162681           1  950430  CREDIT   
           4            4    167083           1  950513  CREDIT   
...                   ...       ...         ...     ...     ...   
           75          75    337454           1  960622   DEBIT   
           76          76    342487           1  960630   DEBIT   
           77          77    343780           1  960630  CREDIT   
           78          78    345046           1  960705   DEBIT   
           79          79    350570           1  960713  CREDIT   

                                  operation   amount  balance  \
account_id                                                      
1          0                 CREDIT IN CASH   1000.0   1000.0   
           1   COLLECTION FROM ANOTHER BANK   3679.0   4679.0   
           2                 CREDIT IN CASH  12600.0  17279.0   
           3                            NaN     19.2  17298.2   
           4   COLLECTION FROM ANOTHER BANK   3679.0  20977.2   
...                                     ...      ...      ...   
           75               CASH WITHDRAWAL    360.0  16692.4   
           76               CASH WITHDRAWAL     14.6  16743.4   
           77                           NaN     65.5  16758.0   
           78    REMITTANCE TO ANOTHER BANK   2452.0  14291.4   
           79  COLLECTION FROM ANOTHER BANK   3679.0  17970.4   

                           k_symbol  age  ...  year dtme  dtme_cat  age_group  \
account_id                                ...                                   
1          0                    NaN   29  ...  1995    7      last      18-30   
           1                    NaN   29  ...  1995   17    middle      18-30   
           2                    NaN   29  ...  1995    7      last      18-30   
           3      INTEREST CREDITED   29  ...  1995    0      last      18-30   
           4                    NaN   29  ...  1995   18    middle      18-30   
...                             ...  ...  ...   ...  ...       ...        ...   
           75                   NaN   29  ...  1996    8      last      18-30   
           76  PAYMENT ON STATEMENT   29  ...  1996    0      last      18-30   
           77     INTEREST CREDITED   29  ...  1996    0      last      18-30   
           78             HOUSEHOLD   29  ...  1996   26     first      18-30   
           79                   NaN   29  ...  1996   18    middle      18-30   

                 td  log_amount  log_amount_sc     td_sc    age_sc  tcode_num  
account_id                                                                     
1          0    0.0    3.000434       2.823750  0.000000  1.745524          0  
           1   20.0    3.565848       3.355869  3.298201  1.745524          1  
           2   10.0    4.100405       3.858949  1.649100  1.745524          0  
           3    7.0    1.305351       1.228484  1.154370  1.745524          2  
           4   13.0    3.565848       3.355869  2.143831  1.745524          1  
...             ...         ...            ...       ...       ...        ...  
           75   6.0    2.557507       2.406906  0.989460  1.745524          3  
           76   8.0    1.193125       1.122866  1.319280  1.745524          4  
           77   0.0    1.822822       1.715483  0.000000  1.745524          2  
           78   5.0    3.389698       3.190092  0.824550  1.745524          5  
           79   8.0    3.565848       3.355869  1.319280  1.745524          1  

[80 rows x 25 columns]